### Imports and Paths

In [ ]:
import cv2               
import numpy as np         
import os                
from random import shuffle 
from tqdm import tqdm     
import tensorflow as tf
import time
import keras
import matplotlib as mlp
import matplotlib.pyplot as plt
import sys
from keras import losses
from keras.models import Model
from keras.layers import *
from keras.engine import Layer
from keras.models import *
import keras.backend as K
from keras.utils import plot_model
from keras.utils import *
import matplotlib as mlp
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator

#Windows
IMG_DIR = '/Users/Niclas/Dropbox/BA/x'
LABEL_DIR = '/Users/Niclas/Dropbox/BA/y'

AUG_IMG_DIR = '/Users/Niclas/Dropbox/BA/AUGX'
AUG_LABEL_DIR = '/Users/Niclas/Dropbox/BA/AUGY'

seed=1

#check if GPU works
#np.set_printoptions(threshold=np.inf)
tf.test.gpu_device_name()

In [ ]:
def uncategorical(one_hot):      #convert one hot array to sparse
    sparse=[]
    for i in range(len(one_hot)):
        if(one_hot[i,0]>one_hot[i,1]):
            sparse.append(0)
        else:
            sparse.append(255)
    sparse=np.array(sparse)
    return sparse

### Generate augmentation Basis

In [ ]:
def process_image_data():
    testing_data = []
    for img in tqdm(os.listdir(IMG_DIR)): #iterate through all images in the folder
        path = os.path.join(IMG_DIR,img) 
        img_num = img.split('.')[0]
        img = np.array(cv2.imread(path)) #read image
            
        path2 = os.path.join(LABEL_DIR,'polyraster-')   
        path2 +=img_num
        path2+= '.png'  
        label =  cv2.imread(path2,cv2.IMREAD_GRAYSCALE)  #read label     

        testing_data.append([image,label]) 
    return testing_data

In [ ]:
data=process_image_data()    #generate data for augmentation

train_x = np.array([i[0] for i in data])    #split data into images and labels
train_y =np.array([i[1] for i in data])

train_y=train_y.reshape(-1,1704,2272,1) #add 4th dimension
print(np.array(train_x).shape)
print(np.array(train_y).shape)

### Augment Images

In [ ]:
images=1000  #amount of augmented images that will be generated

data_gen_args = dict(featurewise_center=True,  #augmentation attributes
                     featurewise_std_normalization=True,
                     rotation_range=20.,
                     width_shift_range=0.02,
                     height_shift_range=0.02,
                     zoom_range=0.01,
                     horizontal_flip=True,
                     fill_mode="constant",
                     cval=0,
                     data_format="channels_last"
                    ) 
image_datagen = ImageDataGenerator(**data_gen_args) 
mask_datagen = ImageDataGenerator(**data_gen_args)
image_datagen.fit(train_x)  #fit the generator on data to use featurewise_center
mask_datagen.fit(train_y)


#saves the defined amount of augmented images to the specified directory
i = 0
for batch in image_datagen.flow(x=train_x, batch_size=1,
                          save_to_dir=AUG_IMG_DIR, save_prefix='Image', save_format='png',seed=seed):   
    i += 1
    if i >= images:
         break  
i = 0
for batch in mask_datagen.flow(x=train_y, batch_size=1, 
                          save_to_dir=AUG_LABEL_DIR, save_prefix='Label', save_format='png',seed=seed):
    i += 1
    if i >= images:
          break 

In [ ]:
def process_augment_data():
    aug_data = []
    for img in tqdm(os.listdir(AUG_IMG_DIR)): #iterate over augmentation folder
        img_num = img.split('_')  
        img_num='_'+img_num[1]+'_'+img_num[2]
               
        path='Image'                        
        path=path+img_num
        path=os.path.join(AUG_IMG_DIR,path)        
        img = cv2.imread(path)   #read in image
        
        image_length=img.shape[0]
        image_width=img.shape[1]    
        half_length=int(image_length/2)
        half_width=int(image_width/2)
                                   
        path_label='Label'+img_num    
        path_label = os.path.join(AUG_LABEL_DIR,path_label)
        label = cv2.imread(path_label,cv2.IMREAD_GRAYSCALE) #read label
   
        label=label.flatten() 
      
        label=keras.utils.to_categorical(label, 2)  #flatten the label to transform it into one hot
        label=label.reshape(1704,2272,2)
                
        image1 = img[:half_length,:half_width]  # cut the images into equally sized parts to reduce memorey usage in the gpu
        image2 = img[:half_length,half_width:]
        image3 = img[half_length:,:half_width]       
        image4 = img[half_length:,half_width:]

        label1 = label[:half_length,:half_width]
        label2 = label[half_length:,:half_width]
        label3 = label[:half_length,half_width:]
        label4 = label[half_length:,half_width:]
        
        aug_data.append([image1,label1]) 
        aug_data.append([image2,label2]) 
        aug_data.append([image3,label3]) 
        aug_data.append([image4,label4]) 
    shuffle(aug_data)
    return aug_data

In [ ]:
data=process_augment_data() #generate augmented data

augment_x = np.array([i[0] for i in data])   #split into images and labels
augment_y = np.array([i[1] for i in data]) 

print(augment_x.shape)
print(augment_y.shape)

train_test_split=2000  #validation split
augment_x_train=augment_x[:train_test_split]
augment_x_test=augment_x[train_test_split:]

augment_y_train=augment_y[:train_test_split]
augment_y_test=augment_y[train_test_split:]

print(augment_x_train.shape)
print(augment_y_train.shape)
print(augment_x_test.shape)
print(augment_y_test.shape)

### Neural Network Defintion

In [ ]:
kernel = 5
pad = 2
pool_size = 2

c=Input(shape=(852,1136,3)) 

a1=ZeroPadding2D(padding=(pad,pad)) (c)  
a2=Conv2D(64, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (a1)
a3=BatchNormalization() (a2)
a4=Activation('relu') (a3)
a5=MaxPooling2D(pool_size=(pool_size, pool_size)) (a4)

b1=ZeroPadding2D(padding=(pad,pad)) (a5) 
b2=Conv2D(128, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (b1)
b3=BatchNormalization() (b2)
b4=Activation('relu') (b3)
b5=MaxPooling2D(pool_size=(pool_size, pool_size)) (b4)

c1=ZeroPadding2D(padding=(pad,pad)) (b5) 
c2=Conv2D(256, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (c1)
c3=Dropout(0.4) (c2)
c4=BatchNormalization() (c3)
c5=Activation('relu') (c4)
c6=MaxPooling2D(pool_size=(pool_size, pool_size)) (c5)

d1=ZeroPadding2D(padding=(pad,pad)) (c6) 
d2=Conv2D(512, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (d1)
d3=Dropout(0.6) (d2)
d4=BatchNormalization() (d3)
d5=Activation('relu') (d4)

e1=ZeroPadding2D(padding=(pad,pad)) (d5) 
e2=Conv2D(512, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (e1)
e3=Dropout(0.6) (e2)
e4=BatchNormalization() (e3)
e5=Activation('relu') (e4)

f1=UpSampling2D(size=(pool_size,pool_size)) (e5)
f2=ZeroPadding2D(padding=(pad,pad)) (f1)
f3=Conv2D(256, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (f2)
f4=Dropout(0.5) (f3)
f5=BatchNormalization() (f4)
f6=Activation('relu') (f5)

merge1=Add()([c5,f6])

g1=UpSampling2D(size=(pool_size,pool_size)) (merge1)
g2=ZeroPadding2D(padding=(pad,pad)) (g1)
g3=Conv2D(128, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (g2)
g4=BatchNormalization() (g3)
g5=Activation('relu') (g4)

merge2=Add()([g5,b4])

h1=UpSampling2D(size=(pool_size,pool_size)) (g5)
h2=ZeroPadding2D(padding=(pad,pad)) (h1)
h3=Conv2D(64, kernel, kernel, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (h2)
h4=BatchNormalization() (h3)
h5=Activation('relu') (h4)

i1=Conv2D(2, 1, 1, border_mode='valid',kernel_initializer=initializers.TruncatedNormal()) (h5)
i2=Activation('softmax') (i1)

model = Model(inputs=c, outputs=i2)
print(model.output_shape)

plot_model(model, to_file='network.png')

In [ ]:
#use adam with a learning rate of 0.01 and decay of 0.001
#use categorical crossentropy for single class classification problem
model.compile(loss="categorical_crossentropy",metrics=['accuracy'], optimizer=optimizers.adam(lr=0.01,decay=0.001))

### Training

In [ ]:
#fit the model with a batchsize of 1 because the memory storage in the gpu is enormous
model.fit(augment_x_train, augment_y_train, batch_size=1, epochs=40,
                    verbose=1,validation_data=(np.array(augment_x_test),np.array(augment_y_test)))

In [ ]:
#save results
model.save_weights('current_model_weights')
model.save('current_model')